**Note:** The test data should be of sufficiently high resolution if the training data is not.

In [ ]:
import math
import time
import cv2
from sklearn import neighbors
import numpy as np
import pandas as pd
import os
import os.path
import requests
import pickle
from PIL import Image, ImageDraw
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder
curd = os.getcwd()
try:  
    os.mkdir("{}/known_people".format(curd))
except:
    pass
try:  
    os.mkdir("{}/models".format(curd))
except:
    pass
try:  
    os.mkdir("{}/test".format(curd))
except:
    pass
try:  
    os.mkdir("{}/unknown_pictures".format(curd))
except:
    pass
names=[]

In [ ]:
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

In [ ]:

def predict(frame, knn_clf=None, model_path=None, distance_threshold=0.6):
    

    if knn_clf is None and model_path is None:
        raise Exception("Must supply knn classifier either thourgh knn_clf or model_path")

    if knn_clf is None:
        with open(model_path, 'rb') as f:
            knn_clf = pickle.load(f)

    
   # X_img = face_recognition.load_image_file(X_img_path)
    X_img = frame
    X_face_locations = face_recognition.face_locations(X_img)

    if len(X_face_locations) == 0:
        return []

    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]

    return [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]



### Generating test data with PhoneCam

In [ ]:
#test Data Creator
import cv2
import numpy as np
import time
curd = os.getcwd()
#name = input()
url = "http://10.4.58.83:8080/shot.jpg"
count=0
while count<10:
    curtime = time.strftime("%Y_%m_%d-%H_%M_%S")
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content),dtype = np.uint8)
    frame = cv2.imdecode(img_arr,-1)
    cv2.imwrite("{}/test/{}.jpg".format(curd,curtime), frame)
    count+=1
    cv2.imshow('img',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    time.sleep(1)    


    
cv2.destroyAllWindows()

### Generating test set with WebCam

In [ ]:
#test Data Creator

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS, 100)
rat, frame = cap.read()
count=0
while count<10:
    curtime = time.strftime("%Y_%m_%d-%H_%M_%S")
    rat, frame = cap.read()
    cv2.imwrite("{}/test/{}.jpg".format(curd,curtime), frame)
    count+=1
    cv2.imshow('img',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    time.sleep(1)    


cap.release()    
cv2.destroyAllWindows()